In [2]:
import os
import sqlite3
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.codeinput import CodeInput
from kivy.uix.button import Button
from kivy.uix.gridlayout import GridLayout
from kivy.uix.label import Label
from kivy.uix.scrollview import ScrollView
from pygments.lexers.sql import SqlLexer

# Deaktiviere das Argument-Parsen von Kivy (wichtig in Jupyter)
os.environ['KIVY_NO_ARGS'] = '1'

# Pfad zur Chinook-Datenbank (anpassen, falls nötig)
DB_PATH = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"

class SQLApp(App):
    # Überschreibe load_kv, damit keine .kv-Datei geladen wird
    def load_kv(self, *args, **kwargs):
        pass

    def build(self):
        self.main_layout = BoxLayout(orientation='vertical', padding=10, spacing=10)
        
        # CodeInput für SQL-Abfragen mit Syntax Highlighting
        self.input = CodeInput(
            lexer=SqlLexer(),
            size_hint=(1, 0.2),
            multiline=True
        )
        self.main_layout.add_widget(self.input)
        
        # Horizontaler Layoutbereich für die Buttons
        button_layout = BoxLayout(size_hint=(1, 0.1), spacing=10)
        
        # Button für das Ausführen der SQL-Abfrage
        execute_btn = Button(text='Abfrage ausführen')
        execute_btn.bind(on_press=self.execute_query)
        button_layout.add_widget(execute_btn)
        
        # Button zum Beenden der App
        exit_btn = Button(text='Beenden')
        exit_btn.bind(on_press=self.stop_app)
        button_layout.add_widget(exit_btn)
        
        self.main_layout.add_widget(button_layout)
        
        # ScrollView für die Ergebnisanzeige als Tabelle
        self.scroll_view = ScrollView(size_hint=(1, 0.7))
        self.result_grid = GridLayout(cols=1, spacing=5, size_hint_y=None)
        self.result_grid.bind(minimum_height=self.result_grid.setter('height'))
        self.scroll_view.add_widget(self.result_grid)
        self.main_layout.add_widget(self.scroll_view)
        
        return self.main_layout

    def clear_table(self):
        # Entfernt alle Widgets aus dem Ergebnisbereich
        self.result_grid.clear_widgets()

    def execute_query(self, instance):
        query = self.input.text.strip()
        if not query:
            self.clear_table()
            self.result_grid.add_widget(Label(text="Bitte gib ein SQL-Statement ein."))
            return
        
        try:
            # Verbindung zur SQLite-Datenbank herstellen und Abfrage ausführen
            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description] if cursor.description else []
            conn.close()
            
            self.clear_table()
            
            if not results:
                self.result_grid.add_widget(Label(text="Abfrage erfolgreich ausgeführt. Keine Ergebnisse."))
                return

            # Erstelle eine Kopfzeile für die Tabelle
            header_layout = GridLayout(cols=len(columns), size_hint_y=None, height=40, padding=5)
            for col in columns:
                header_layout.add_widget(Label(text=str(col), bold=True))
            self.result_grid.add_widget(header_layout)
            
            # Füge für jede Datenzeile eine Zeile im GridLayout hinzu
            for row in results:
                row_layout = GridLayout(cols=len(columns), size_hint_y=None, height=30, padding=5)
                for cell in row:
                    row_layout.add_widget(Label(text=str(cell)))
                self.result_grid.add_widget(row_layout)
            
        except sqlite3.Error as e:
            self.clear_table()
            self.result_grid.add_widget(Label(text=f"Fehler bei der Abfrage: {e}"))

    def stop_app(self, instance):
        # Beendet die App
        App.get_running_app().stop()

if __name__ == '__main__':
    SQLApp().run()


In [ ]:
import os
import sqlite3
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.codeinput import CodeInput
from kivy.uix.button import Button
from kivy.uix.gridlayout import GridLayout
from kivy.uix.label import Label
from kivy.uix.scrollview import ScrollView
from pygments.lexers.sql import SqlLexer

# Deaktiviere das Argument-Parsen von Kivy (wichtig in Jupyter)
os.environ['KIVY_NO_ARGS'] = '1'

# Pfad zur Chinook-Datenbank (anpassen, falls nötig)
DB_PATH = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"

class SQLApp(App):
    # Überschreibe load_kv, damit keine .kv-Datei geladen wird
    def load_kv(self, *args, **kwargs):
        pass

    def build(self):
        self.main_layout = BoxLayout(orientation='vertical', padding=10, spacing=10)
        
        # CodeInput für SQL-Abfragen mit Syntax Highlighting
        self.input = CodeInput(
            lexer=SqlLexer(),
            size_hint=(1, 0.2),
            multiline=True
        )
        self.main_layout.add_widget(self.input)
        
        # Horizontaler Layoutbereich für die Buttons
        button_layout = BoxLayout(size_hint=(1, 0.1), spacing=10)
        
        # Button für das Ausführen der SQL-Abfrage
        execute_btn = Button(text='Abfrage ausführen')
        execute_btn.bind(on_press=self.execute_query)
        button_layout.add_widget(execute_btn)
        
        # Button zum Beenden der App
        exit_btn = Button(text='Beenden')
        exit_btn.bind(on_press=self.stop_app)
        button_layout.add_widget(exit_btn)
        
        self.main_layout.add_widget(button_layout)
        
        # ScrollView für die Ergebnisanzeige als Tabelle
        self.scroll_view = ScrollView(size_hint=(1, 0.7))
        self.result_grid = GridLayout(cols=1, spacing=5, size_hint_y=None)
        self.result_grid.bind(minimum_height=self.result_grid.setter('height'))
        self.scroll_view.add_widget(self.result_grid)
        self.main_layout.add_widget(self.scroll_view)
        
        return self.main_layout

    def clear_table(self):
        # Entfernt alle Widgets aus dem Ergebnisbereich
        self.result_grid.clear_widgets()

    def execute_query(self, instance):
        query = self.input.text.strip()
        if not query:
            self.clear_table()
            self.result_grid.add_widget(Label(text="Bitte gib ein SQL-Statement ein."))
            return
        
        try:
            # Verbindung zur SQLite-Datenbank herstellen und Abfrage ausführen
            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description] if cursor.description else []
            conn.close()
            
            self.clear_table()
            
            if not results:
                self.result_grid.add_widget(Label(text="Abfrage erfolgreich ausgeführt. Keine Ergebnisse."))
                return

            # Erstelle eine Kopfzeile für die Tabelle
            header_layout = GridLayout(cols=len(columns), size_hint_y=None, height=40, padding=5)
            for col in columns:
                header_layout.add_widget(Label(text=str(col), bold=True))
            self.result_grid.add_widget(header_layout)
            
            # Füge für jede Datenzeile eine Zeile im GridLayout hinzu
            for row in results:
                row_layout = GridLayout(cols=len(columns), size_hint_y=None, height=30, padding=5)
                for cell in row:
                    row_layout.add_widget(Label(text=str(cell)))
                self.result_grid.add_widget(row_layout)
            
        except sqlite3.Error as e:
            self.clear_table()
            self.result_grid.add_widget(Label(text=f"Fehler bei der Abfrage: {e}"))

    def stop_app(self, instance):
        # Beendet die App
        App.get_running_app().stop()

if __name__ == '__main__':
    SQLApp().run()


: 